### 一、数据转换 Date Transformation & Date Massage 的 定义

##### Date Transformation ： 基于以下三个原因 在完成 数据清洗 数据集成 数据归约后必须去做的 一个预处理步骤    
1. 必要性(necessity):不转换无法使用的，比如文本类转化数值类。典型的有模型KMEANS KNN 等计算样本距离的算法和MLP多层感知器 Multi-Layer Perceptron    
2. 正确性(correctness):不转换导致结果出错的，比如数值类完成归一化。典型的有 模型KMEANS KNN 等计算样本距离 的 必须保证每个特征的取值范围一致 避免权重不等     
3. 有效性(effectiveness):转化后会使分析更为有效     

##### Date Massage: Date Transformation 必定包含 Date Massage 但不一定是 Date Massage ；Date Massage 则一定是 Date Transformation    
1. 原因在于：数据按摩有正面描述感，代表做得多就效果好 因此他主要着重于 数据转换中的有效性      

### 二、归一化 MinMaxScaler & 标准化 StandardScaler

##### 归一化 
1. 公式：(Xi - X.min() ) / ( X.max() - X.min() )      
2. 特性：数据集归一化后的取值范围一般固定在 [0,1] 可以通过调整公式使得取值范围改变为[-1,1] [0,2] ；因其强制压缩会改变原始数据集的分布特征 ；对于离群值敏感 挤压正常数据
3. 适用于所有距离算法       

##### 标准化
1. 公式：( Xi - X.mean() ) / X.std()     
2. 特性：数据集标准化后会成为方差为1 均值为0 的数据集； 但是不会改变数据的分布特征， 是一种缩放操作 ；对于离群值不敏感
3. 适用于期望保存分布 有过多不可删除的敏感值 或者 pca这种对于方差值有要求的场景

##### 数据转化侧重点为 正确性

In [ ]:
#### 下面演示 下 改变 minmax 的范围为 【-1 ， 1 】的过程
import numpy as np
x = np.random.uniform(5, 100, 20)

In [ ]:
x

In [ ]:
x_minmax = (x - x.min() ) / ( x.max() - x.min() ) 
print( x_minmax.min(),x_minmax.max() )

In [ ]:
x_minmax_1 = 2 * x_minmax - 1
print( x_minmax_1.min(),x_minmax_1.max() )

### 三、分类型数据和数值类数据的转化--大部分为必要性 少部分为有效性 

#### 字符串类型的 特征可以被分类为 标称特征或者序数特征 
#### 对于标称特征 建议采取如下转化方式
##### 独热编码
1. 将分类特征转化为独立的二进制向量 如分类特征枚举值为ABCDE 则转化为 A[1,0,0,0,0] B[0,1,0,0,0] C[0,0,1,0,0] D[0,0,0,1,0] 3[0,0,0,0,1]
2. 优点：不引入任何数值大小相关信息 避免误导模型认为 标称特征的枚举值存在大小关系 
3. 缺点：维度爆炸 稀疏性提高 影响模型性能
4. 适用于：非序数特征 且枚举值 不多的情况
5. pandas中可采取 ： pd.get_dummies()完成独热编码

##### 频率编码
1. 将分类特征 换为 类别出现的频率 如 红色出现100次，绿色出现50次，蓝色出现20次 → 红色 → 100，绿色 → 50，蓝色 → 20。
2. 优点：对于高基数类别特征时采取频率编码效果很好
3. 缺点：对于数据集样本不够的情况下 可能导致过拟合 因为小数据的频率和大数据的频率可能存在显著差异
4. 适用于：遭遇高基数类别特征时 使用这个 否则最好不用这个


In [ ]:
#### 示例独热编码 One-Hot Encoding
import pandas as pd
import numpy as np
df = pd.read_csv(r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\Hands-On-Data-Preprocessing-in-Python-main\Chapter14\WH Report_preprocessed.csv")
print(f'对于此数据集中的Continent特征进行独热编码，其枚举值为: {df.Continent.unique()}，其个数为{len(df.Continent.unique())}') 

## 完成独热编码
Continent_One_Hot = pd.get_dummies(df.Continent)

## 对其他要使用的数值类特征进行归一化处理
dimensions = ['Life_Ladder',
       'Log_GDP_per_capita', 'Social_support',
       'Healthy_life_expectancy_at_birth', 'Freedom_to_make_life_choices',
       'Generosity', 'Perceptions_of_corruption', 'Positive_affect',
       'Negative_affect']
Xs = df[dimensions]
Xs_minmax = ( Xs - Xs.min() ) / ( Xs.max() - Xs.min() )
Xs_union = Xs_minmax.join(Continent_One_Hot/7) # 一个特征演化为7个特征 将会导致权重变更 所以要除以特征数还原 另外不要和数值类一起归一化会导致/7被取消

## 进行聚类 set OMP_NUM_THREADS=5 注意先在jupyter notebook中运行下这个语句避免内存泄漏
from sklearn.cluster import KMeans
n_clusters = 3
kmeans = KMeans(n_clusters = n_clusters,n_init  = 10)
kmeans.fit(Xs_union)

## 查看质心  
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
sns.heatmap(kmeans.cluster_centers_,annot= True,cmap='Greys',)
plt.xticks( np.array(range(len(Xs_union.columns))) ,list(Xs_union.columns),rotation = 90)
plt.show()

#### 对于序数特征 建议采取如下转化方式
##### 排序转换
1. 将分类特征排序之后转化为有大有小的数字 比如小学 中学 高中 大学 转化为 0,1,2,3
2. 优点：保留了分类本身的序数性质
3. 缺点：假设了每个等级的学历是等差值为1的，掺杂了本身不包含的信息
4. 适用于：序数特征 保留序数特性

##### 特性构造
1. 将分类特征排序之后转化为有大有小的数字 比如小学 中学 高中 大学 转化为 0,1,2,3     
2. 对于这个例子 特性构造的逻辑是通过每个学历的年级来转换为数字 比如大学是要17年读完 那就转换为17的数值     
3. 此处仅仅提及特性构造 其复杂性让我们单独开一个章节 见下     

#### 对于数值特征 建议采取如下转化方式
##### 离散化
1. 将数值特征 换为 分类特征 如 连续的工作年龄 0-20 我等分他为 0-5年的 5-10年 10-15年的 15-20年的 就变成分类特征      
2. 优点：可以使得分析生成的输出更为直观和好理解
3. 缺点：无显著缺点 但是如何分界需要你自行探索数据
4. 适用于：需要便于理解的时候 改数字为分类    

In [ ]:
#### 示例离散化
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## 获取数据
df = pd.read_csv(r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\Hands-On-Data-Preprocessing-in-Python-main\Chapter14\adult.csv")
print(f'对于此数据集中 每周工作时间 离散化') 

## 准备绘图区域
figure,axs = plt.subplots(3,1,figsize = (20,10))
ax1,ax2,ax3 = axs.flatten()

## 检查需要离散化的数据分布情况
df.hoursPerWeek.plot.hist(ax = ax1,color = 'r',alpha = 0.5)   

## 基于数据分布 认定40是合格的分界线 后续会有更多 分区方法的介绍 准备1个离散函数并应用
def hoursPerWeek_to_string(hoursPerWeek):
    if hoursPerWeek >40:
        string = 'hoursPerWeek > 40'
    elif hoursPerWeek == 40:
        string = 'hoursPerWeek = 40'
    elif hoursPerWeek < 40:
        string = 'hoursPerWeek <= 40'
    return string
df['dis_hoursPerWeek'] = df.hoursPerWeek.apply(hoursPerWeek_to_string)   

## 对离散化后的数据执行绘图
dfagg = df.groupby(['sex','income','dis_hoursPerWeek']).agg(count = ('age','count')).unstack()  # unstack() 是为了增加一个维度 
dfagg.plot.barh(color = ['y','b','r'],ax = ax2)   # x是数值 y（'sex','income'）是分类 被推出去的dis_hoursPerWeek 则成为颜色维度

## 对比如果用箱型图绘画 三者的关系 可以发现很难理解 
sns.boxplot(data = df,y='sex',x = 'hoursPerWeek',hue = 'income',ax = ax3)

In [ ]:
#### 了解离散化的类型
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
df = pd.read_csv(r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\Hands-On-Data-Preprocessing-in-Python-main\Chapter14\adult.csv")
## 均匀分布数据可用等宽法，不均匀可用等频法或聚类。 
## 等宽（equal width）
pd.cut(df.age,bins =5).value_counts().sort_index().plot.bar()
## 等频 (equal frequency)
pd.qcut(df.age,q =5).value_counts().sort_index().plot.bar()
## 点对点(ad hoc)
# 基于有关特性的相关知识来规定分界点 比如基于我们知道美国大多数员工每周工作40小时的间接知识 来确定40是个合适的分界点

#### 了解离散化分界点的数量
## 在数据值离散化的期间中，k个分界点 将带来 k+1个分类枚举值
## 通过直方图探索 或者结合领域的专业知识 去做判断

### 四、特性构造

In [ ]:
##### 前文解释了可以通过单个特征 的认知 构造一个新的特征 本处 再展示一个 对多个特征的认知 构成一个新的特征
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## 准备绘图的数据
df = pd.read_csv(r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\Hands-On-Data-Preprocessing-in-Python-main\Chapter14\500_Person_Gender_Height_Weight_Index.csv")
df.Index = df.Index.replace({0:'Extremely Weak',1:'Weak',2:'Normal',3:'Overweight',4:'obesity',5:'Extremely obesity'})
df = df.rename(columns = {'Index':'Condition'})
df['BMI'] = df.apply(lambda r : r.Weight / (r.Height/100) ** 2 , axis = 1)  # 这是个BMI专业的计算公式  Height/100 是换算为米 单位 然后 Weight 是公斤单位
plt.figure(figsize = (20,10))

## 准备绘图区域
figure,axs = plt.subplots(4,1,figsize = (20,10))
ax1,ax2,ax3,ax4 = axs.flatten()

## 专业的医生通过绘画散点图 发现 体重和身高 两者的比值 一定程度将 身体状况 分割出清晰的组别
sns.scatterplot(data = df, x= 'Height',y = 'Weight',hue = 'Condition',style = 'Gender', ax = ax1)

## 专业人士知道的BMI指标，上文已经计算 我们可以绘画图 比较 健康状况和bmi数值之间的关系 
sns.scatterplot(data = df, x= 'BMI',y = 1 , hue = 'Condition', ax = ax2)  # y没有实际意义 只是为了展示出点

## 观察图2可以发现出规律了 但是看起来不美观 引入一种名为抖动的技术 在这里就是为了让y=1 变成y = 随机浮点数  从而在y周上面 分离开来点 抖动开来点
df['Random'] = np.random.random(len(df))  # 此步骤是为了创建一个抖动 一个抖动 就可以 做到比原先的散点图要分离点 方便观察相关性
sns.scatterplot(data = df, x= 'BMI',y = 'Random',hue = 'Condition',ax =ax3)

## 那么为什么不能是身高/体重？ 事实证明也是可以的 但是能看出来区分程度 不如 bmi这个专业指标
df['height/weight'] =   df.apply(lambda r : r.Weight / r.Height, axis = 1) 
sns.scatterplot(data = df, x= 'height/weight',y = 'Random',hue = 'Condition',ax =ax4)

### 五、特征提取

In [ ]:
# 一个特性提取三个特性 比如邮箱地址 拆分为 流行的免费平台  .edu是否教育域名 和 是否特殊域名三个特征
# 或者形态特征提取 对于汽车发动机发出的频率  提取为波峰 波谷 和最大振幅3个特征 
# 或者在之前的二级清洗中的解包操作 提取长文本中的 关键字 出现的频率
# 或者在降维中的FDA 或者 PCA 也可以算作特征提取

In [ ]:
#### 实战案例 提取下 汽车发动机发出的频率  提取为波峰 波谷 和最大振幅3个特征 
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier,export_graphviz
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import scipy.signal as signal

### 读取数据 获取基础特征
df = pd.read_csv(r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\Noise_Analysis.csv")
Xs = df.drop(['index','Label'],axis = 1)
y  = df.Label

### 对于基础特征上 做特征构造 
distance = 15 # 观察周期为大约30 分为波峰和波谷 我取一半

### 波峰 波谷数量  高点低点距离差 正信号总和 负信号总和
Xss['n_peaks'] = Xs.apply(lambda row : len(signal.find_peaks(row,distance = distance)[0]) , axis = 1)
Xss['n_valleys'] = Xs.apply(lambda row : len(signal.find_peaks(-row,distance = distance)[0]) , axis = 1)
Xss['max_oscilate'] = Xs.apply(max,axis = 1) - Xs.apply(min,axis = 1)
Xss['positive_sign'] = Xs.apply(lambda row : abs(row[row > 0].sum()) , axis = 1)
Xss['negative_sign'] = Xs.apply(lambda row : abs(row[row < 0].sum()) , axis = 1)

### 架构模型拟合
tree = DecisionTreeClassifier()
tree.fit(Xss,y)

dot = export_graphviz(tree,feature_names = Xss.columns, class_names=tree.classes_, filled = True, rounded = True, special_characters =True)
picture = graphviz.Source(dot)
picture.render(r"C:\Users\yaoyan\Downloads\权益说明",format ='png')

### 六、对数变化

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math

## 读取样本数据 并观察其为指数增长 意识到异常值非常多 不利于模型 
df = pd.read_csv(r"D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\Hands-On-Data-Preprocessing-in-Python-main\Chapter14\GDP 2019 2020.csv")
df.sort_values('2020',inplace = True)
figure,axs = plt.subplots(3,1,figsize=(20,10))
ax1,ax2,ax3 = axs.flatten()
df['2020'].plot.box(vert = False , ax = ax1)
ax2.plot(df['Country Name'],df['2020'])
plt.sca(ax2) #手动切换子图
plt.xticks(rotation = 90)


## 对于数据做出对数转换
df['2020log'] = np.log(df['2020'])
ax3.plot(df['Country Name'],df['2020log'])
plt.sca(ax3) 
plt.xticks(rotation = 90)
plt.show()

## 部分写法中提供了对数转换的参数 比如
# df['2020'].sort_values().plot(logy = True) 建议还是你有意识的手动

### 七、平滑 聚合 分箱

In [ ]:
import matplotlib.pyplot as plt

def show_color(rgb):
    plt.imshow([[rgb]])
    plt.axis('off')
    plt.show()

show_color((255, 0, 0))  # 显示红色


def generate_gradient(color1, color2, steps):
    gradient = []
    for i in range(steps):
        r = int(color1[0] + (color2[0] - color1[0]) * i / (steps - 1))
        g = int(color1[1] + (color2[1] - color1[1]) * i / (steps - 1))
        b = int(color1[2] + (color2[2] - color1[2]) * i / (steps - 1))
        gradient.append((r, g, b))
    return gradient

# 生成从红色到蓝色的渐变
gradient = generate_gradient((255, 0, 0), (0, 0, 255), 10)

# 显示渐变颜色
for color in gradient:
    show_color(color)

In [ ]:
import matplotlib.colors as mcolors

# 获取所有预定义颜色的名称和对应的 RGB 值
colors = mcolors.CSS4_COLORS

# 打印一些常见颜色的名称和 RGB 颜色编码
for color_name, color_code in list(colors.items())[:10]:  # 只显示前10个
    print(f"{color_name}: {color_code} (RGB: {mcolors.to_rgb(color_code)})")

### 八、心里问题研究案例

In [ ]:
# 数据来源：https://osmihelp.org/research.html
# Kaggle Hub 是一个由 Kaggle 提供的平台，旨在为数据科学家和机器学习爱好者提供一个集成的环境，以便于学习、分享和合作。 

In [ ]:
import kagglehub
path = kagglehub.dataset_download("osmihelp/osmi-mental-health-in-tech-survey-2019")
print("Path to dataset files:", path)

In [51]:
import pathlib
import pandas as pd
path = r'D:\1-script\3-PYTHON\data存储jupyter用的个文件类型数据\kagglehub\osmihelp'
path_object = pathlib.Path(path)
df_dict = {}
q_set = set()
for csv_path in path_object.rglob('*.csv'):
    year = str(csv_path.parent).split('\\')[-1]
    df_dict[year]  = pd.read_csv(csv_path)
    q_set.update(df_dict['2016'].columns)

q_dict = { q:'Q'+ str(i) for i,q in enumerate(q_set ,start = 1) }

for df in df_dict.values():
    print('切换一个df')
    for column in df.columns:
        print(column)
        print(q_dict[column])
    [ q_dict[column]  for column in df.columns]

切换一个df
Are you self-employed?
Q41
How many employees does your company or organization have?
Q19
Is your employer primarily a tech company/organization?
Q6
Is your primary role within your company related to tech/IT?
Q31
Does your employer provide mental health benefits as part of healthcare coverage?
Q47
Do you know the options for mental health care available under your employer-provided coverage?
Q14
Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?
Q44
Does your employer offer resources to learn more about mental health concerns and options for seeking help?
Q23
Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?
Q16
If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:
Q45
Do you think that discussing a mental health disorder with your employer would have

KeyError: '#'

In [53]:
df_dict['2017']

,#,<strong>Are you self-employed?</strong>,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,...,What is your gender?,What country do you <strong>live</strong> in?,What US state or territory do you <strong>live</strong> in?,What is your race?,Other.3,What country do you <strong>work</strong> in?,What US state or territory do you <strong>work</strong> in?,Start Date (UTC),Submit Date (UTC),Network ID
0,e49fe87572831232dcfa51b376b22039,0,100-500,1.0,1.0,No,Yes,No,I don't know,I don't know,...,Female,United Kingdom,NaN,NaN,NaN,United Kingdom,NaN,2018-05-16 12:32:04,2018-05-16 12:42:40,464b7a12f1
1,a1eede444ac024928d247a8372d54931,0,100-500,1.0,1.0,Yes,Yes,No,No,I don't know,...,male,United Kingdom,NaN,NaN,NaN,United Kingdom,NaN,2018-05-16 12:31:13,2018-05-16 12:40:40,464b7a12f1
2,37d3fd67f62bd1e0a2dea4f9cd440d98,0,6-25,1.0,1.0,I don't know,No,I don't know,No,Yes,...,male,United States of America,Missouri,White,NaN,United States of America,Missouri,2018-05-09 05:34:05,2018-05-09 05:46:04,1eb7e0cb94
3,519b759442c1cab0e9b5a8a1acb1b216,0,More than 1000,1.0,1.0,Yes,Yes,I don't know,I don't know,Yes,...,Male,United States of America,Washington,White,NaN,United States of America,Washington,2018-05-04 23:19:14,2018-05-04 23:23:23,63852edbc4
4,ef0af4927b575b1a3e607c11ca37870e,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,female,United States of America,Illinois,More than one of the above,NaN,United States of America,Illinois,2018-05-03 00:40:24,2018-05-03 00:53:20,43237889f1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,97935fe0d63490dfcff17d405d06e579,0,26-100,1.0,1.0,I don't know,No,No,I don't know,Yes,...,Male,United Kingdom,NaN,NaN,NaN,United Kingdom,NaN,2017-10-19 12:44:45,2017-10-19 12:51:49,addd5ce59b
752,2233fdfc9a4bd123359cafbfc8f53892,0,6-25,1.0,1.0,Yes,No,I don't know,No,Yes,...,Male,United States of America,Texas,White,NaN,United States of America,Texas,2017-09-06 13:03:08,2017-09-06 13:13:09,d345b64d29
753,acff6941d3e2683957e875cc1ad4dbe6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-31 18:05:07,2017-08-31 18:06:56,bae691937c
754,21a68c42a608a03d541600176acc1d9a,0,26-100,1.0,1.0,Yes,Yes,No,No,No,...,dude,United States of America,Tennessee,White,NaN,United States of America,Tennessee,2017-08-31 13:44:22,2017-08-31 13:55:29,d45999fdbd


{'Q1': 'If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?',
 'Q2': 'If maybe, what condition(s) do you believe you have?',
 'Q3': 'Would you be willing to bring up a physical health issue with a potential employer in an interview?',
 'Q4': 'Do you know local or online resources to seek help for a mental health disorder?',
 'Q5': 'Have you had a mental health disorder in the past?',
 'Q6': 'Is your employer primarily a tech company/organization?',
 'Q7': 'Do you think that discussing a mental health disorder with previous employers would have negative consequences?',
 'Q8': 'If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?',
 'Q9': 'What US state or territory do you work in?',
 'Q10': 'Would you have been willing to discuss a mental health issue with your previous co-workers?',
 'Q11': 'If so, what condition(s) were you diagnosed with?',
